<a href="https://colab.research.google.com/github/MonalSD/ML/blob/main/NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [24]:
def fit(x_train,y_train):
  result ={}
  class_values = set(y_train)
  for current_class in class_values:
    result[current_class] = {}
    result["total_data"] = len(y_train)
    current_class_rows = (y_train == current_class)
    x_train_current = x_train[current_class_rows]
    y_train_current = y_train[current_class_rows]
    num_feature = x_train.shape[1]
    result[current_class]["total_count"] = len(y_train_current)
    for j in range(1,num_feature+1):
      result[current_class][j] ={}
      all_possible_values = set (x_train[:,j-1])
      for current_value in all_possible_values:
        result[current_class][j][current_value] =  (x_train_current[:,j-1] == current_value).sum()

    return result

In [25]:
def probability(dictionary,x,current_class):
  output = np.log(dictionary[current_class]["total_count"])-np.log(dictionary["total_data"])
  num_features = len(dictionary[current_class].keys())-1
  for j in range(1,num_features+1):
    xj = x[j-1]
    count_current_class_with_value_xj = dictionary[current_class][j][xj] +1
    count_current_Class = dictionary[current_class]["total_count"] + len(dictionary[current_class][j].keys())
    current_xj_probability = np.log(count_current_class_with_value_xj)-np.log(count_current_Class)
    output = output + current_xj_probability

  return output


In [26]:
def predictSinglePoint(dictionary,x):
  classes = dictionary.keys()
  best_p = -1000
  best_class = -1
  first_run = True
  for current_class in classes:
    if(current_class == "total_data"):
       continue
    p_current_class = probability(dictionary,x,current_class)
    if (first_run or p_current_class > best_p):
      best_p = p_current_class
      best_class = current_class
    first_run = False

  return best_class


In [27]:
def predict(dictionary,x_test):
  y_pred = []
  for x in x_test:
    x_class = predictSinglePoint(dictionary,x)
    y_pred.append(x_class)
    
  return y_pred

In [28]:
def makeLabelled(column):
  second_limit = column.mean()
  first_limit = 0.5*second_limit
  third_limit = 1.5*second_limit
  for i in range (0,len(column)):
    if(column[i]<first_limit):
      column[i] =0
    elif(column[i]<second_limit):
      column[i] =1
    elif(column[i]<third_limit):
      column[i] =2
    else:
      return 3
  return column

In [29]:
from sklearn import datasets
iris = datasets.load_iris()
x= iris.data
y = iris.target

In [30]:
for i in range(0,x.shape[-1]):
  x[:,i] = makeLabelled(x[:,i])

In [31]:
from sklearn import model_selection
x_train,x_test,y_train,y_test = model_selection.train_test_split(x,y,test_size =0.25,random_state=0)

In [32]:
dictionary = fit(x_train,y_train)

In [33]:
y_pred = predict(dictionary,x_test)

In [36]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.34      1.00      0.51        13
           1       0.00      0.00      0.00        16
           2       0.00      0.00      0.00         9

    accuracy                           0.34        38
   macro avg       0.11      0.33      0.17        38
weighted avg       0.12      0.34      0.17        38

[[13  0  0]
 [16  0  0]
 [ 9  0  0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
classes = dictionary.keys()